## Design and Hedging of a Structured Product

### Experiment 01: Data and Pre-processing

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import os

In [ ]:
class GetStockData:
    def __init__(self, ticker=None, ref_date=None):
        """
        Initialize the GetCostcoData object.

        Args:
            ticker (str): Ticker symbol for the stock (default: "COST").
            ref_date (str): Reference date in 'YYYY-MM-DD' format.
        """
        self.ticker = ticker
        self.ref_date = pd.to_datetime(ref_date)
        self.stock = yf.Ticker(self.ticker)

    def get_historical_data(self, lookback_days=365):
        """
        Fetch historical adjusted close prices up to the reference date.

        Args:
            lookback_days (int): Number of days to look back from the reference date.

        Returns:
            pd.DataFrame: DataFrame with OHLC, Adj Close (if available), and Volume.
        """
        end_date = self.ref_date
        start_date = end_date - timedelta(days=lookback_days)
        df = self.stock.history(
            start=start_date.strftime("%Y-%m-%d"),
            end=end_date.strftime("%Y-%m-%d"),
        )

        # Dynamically select columns that exist
        expected_cols = ["Open", "High", "Low", "Close", "Adj Close", "Volume"]
        available_cols = [col for col in expected_cols if col in df.columns]

        if df.empty:
            raise ValueError("No historical data found for the specified date range.")

        return df[available_cols]

    def get_current_price(self):
        """
        Get the spot price of the stock as of the reference date.

        Returns:
            float: Closing price on the reference date.

        Raises:
            ValueError: If no price data is available for the reference date.
        """
        df = self.stock.history(
            start=self.ref_date.strftime("%Y-%m-%d"),
            end=(self.ref_date + timedelta(days=1)).strftime("%Y-%m-%d"),
        )
        if df.empty:
            raise ValueError("No price data for the specified date.")
        return df["Close"].iloc[0]

    def get_option_chain(self):
        """
        Fetch the option chain closest to the maturity
        (≈ 3 months from the reference date).

        Returns:
            dict: Dictionary with 'calls', 'puts', and 'expiry'.
        """
        option_dates = self.stock.options
        maturity_target = self.ref_date + timedelta(days=91)
        closest_date = min(
            option_dates,
            key=lambda x: abs(pd.to_datetime(x) - maturity_target),
        )
        opt_chain = self.stock.option_chain(closest_date)
        return {
            "calls": opt_chain.calls,
            "puts": opt_chain.puts,
            "expiry": closest_date,
        }

In [3]:
ticker = "COST"
ref_date = "2025-04-01"

stock_data = GetStockData(ticker=ticker, ref_date=ref_date)

In [4]:
historical_df = stock_data.get_historical_data(lookback_days=365)
historical_df = historical_df.reset_index()
historical_df['Date'] = historical_df['Date'].dt.date
print(historical_df.columns)
print(historical_df.shape)
historical_df.tail()

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
(251, 6)


,Date,Open,High,Low,Close,Volume
246,2025-03-25,926.630005,932.289978,922.489990,930.260010,1623000
247,2025-03-26,934.330017,939.419983,923.450012,929.750000,1644700
248,2025-03-27,929.559998,942.900024,928.000000,938.750000,1547000
249,2025-03-28,937.500000,938.750000,926.000000,929.659973,2100500
250,2025-03-31,922.359985,995.000000,922.039978,945.780029,3102000


In [5]:
spot = stock_data.get_current_price()
spot_price_ref_date = pd.DataFrame(data={'ref_date':ref_date, 'spot_price_ref_date':[spot]})
print(f"Spot price on {ref_date}: {spot:.2f} USD")
spot_price_ref_date.head()

Spot price on 2025-04-01: 954.40 USD


,ref_date,spot_price_ref_date
0,2025-04-01,954.400024


In [6]:
option_chain = stock_data.get_option_chain()

calls_df = option_chain["calls"]
puts_df = option_chain["puts"]
expiry_date = option_chain["expiry"]

print(f"Closest option expiry date: {expiry_date}")
print(calls_df.shape)
calls_df.head()

Closest option expiry date: 2025-06-20
(180, 14)


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,COST250620C00255000,2025-04-04 19:11:51+00:00,255.0,665.60,745.35,752.45,0.0,0.0,1.0,5.0,1.400394,True,REGULAR,USD
1,COST250620C00265000,2024-12-16 05:01:17+00:00,265.0,410.00,438.00,447.00,0.0,0.0,NaN,2.0,0.000010,True,REGULAR,USD
2,COST250620C00270000,2023-11-15 15:35:04+00:00,270.0,346.28,399.00,407.75,0.0,0.0,4.0,68.0,0.000010,True,REGULAR,USD
3,COST250620C00280000,2023-12-19 20:34:03+00:00,280.0,410.00,0.00,0.00,0.0,0.0,4.0,2.0,0.000010,True,REGULAR,USD
4,COST250620C00285000,2024-12-16 05:01:17+00:00,285.0,394.00,407.00,416.00,0.0,0.0,NaN,2.0,0.000010,True,REGULAR,USD


In [7]:
print(puts_df.shape)
puts_df.head()

(169, 14)


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,COST250620P00255000,2025-03-19 13:47:13+00:00,255.0,0.53,0.0,0.48,0.0,0.0,1.0,223,1.343753,False,REGULAR,USD
1,COST250620P00265000,2025-03-06 18:14:30+00:00,265.0,0.15,0.0,2.67,0.0,0.0,20.0,33,1.597902,False,REGULAR,USD
2,COST250620P00270000,2023-12-22 18:34:30+00:00,270.0,1.39,0.0,0.00,0.0,0.0,5.0,169,0.500005,False,REGULAR,USD
3,COST250620P00275000,2025-03-06 18:13:57+00:00,275.0,0.04,0.0,2.87,0.0,0.0,1.0,28,1.570559,False,REGULAR,USD
4,COST250620P00280000,2023-12-21 14:30:54+00:00,280.0,4.00,0.0,0.00,0.0,0.0,1.0,9,0.500005,False,REGULAR,USD


### No-Arbitrage Cleaning Summary

We performed systematic cleaning of option data (calls and puts) dated **April 1, 2025** before using it for Bates model calibration. The goal was to ensure the dataset is arbitrage-free, internally consistent, and based on liquid quotes.

---

#### **1. Midpoint Pricing**
- We computed the mid-price as:
  $$
  \text{mid} = \frac{\text{bid} + \text{ask}}{2}
  $$
- If bid and ask were both zero, we fallback to using `lastPrice`.

---

#### **2. Put-Call Parity Check (Relative Error)**
- For strikes where both call and put had non-zero `volume` or `openInterest`, we enforced put-call parity:
  $$
  C - P \approx S_0 e^{-qT} - K e^{-rT}
  $$
- We defined a **relative error**:
  $$
  \text{rel\_error} = \frac{|C - P - (S_0 e^{-qT} - K e^{-rT})|}{|S_0 e^{-qT} - K e^{-rT}| + 1e^{-6}}
  $$
- A tolerance of **15%** was allowed for deviations.

---

#### **3. Monotonicity Check**
- Option prices should behave monotonically:
  - **Calls**: decreasing in strike
  - **Puts**: increasing in strike
- We allowed a small tolerance (`epsilon = 0.5`) to absorb market noise.

---

#### **4. Butterfly Arbitrage Check**
- For three consecutive strikes $(K_1, K_2, K_3)$, we enforced:
  $$
  P(K_2) \leq \frac{1}{2}[P(K_1) + P(K_3)]
  $$
- This avoids convexity violations in the option price curve.

---

#### Final Cleaning Logic
A strike was marked as **clean** if it satisfied:
- Put-call parity (relative error < 15%)
- Monotonicity (within epsilon)
- Butterfly arbitrage (no violation)

We retained:
- **14 clean call strikes**
- **21 clean put strikes**

These are used for Bates model calibration and valuation.

In [8]:
S0 = spot                   # from your saved spot price
r = 0.04                    # risk-free rate
q = 0.008                   # dividend yield
T = 0.25                    # time to maturity
epsilon = 0.5              # acceptable monotonicity deviation (USD)

# --- 1. Compute mid prices ---
calls_df["mid"] = (calls_df["bid"] + calls_df["ask"]) / 2
puts_df["mid"] = (puts_df["bid"] + puts_df["ask"]) / 2

# Fallback: use lastPrice where bid/ask midpoint is 0
calls_df["mid"] = np.where(calls_df["mid"] == 0, calls_df["lastPrice"], calls_df["mid"])
puts_df["mid"] = np.where(puts_df["mid"] == 0, puts_df["lastPrice"], puts_df["mid"])

# --- 2. Put-Call Parity with adaptive tolerance ---
merged_df = calls_df.merge(puts_df, on="strike", suffixes=("_call", "_put"))

# --- Step 3: Filter merged_df by liquidity ---
merged_df = merged_df[
    ((merged_df["volume_call"] > 0) | (merged_df["openInterest_call"] > 0)) &
    ((merged_df["volume_put"] > 0) | (merged_df["openInterest_put"] > 0))
].copy()

merged_df["lhs"] = merged_df["mid_call"] - merged_df["mid_put"]
merged_df["rhs"] = S0 * np.exp(-q * T) - merged_df["strike"] * np.exp(-r * T)
merged_df["abs_diff"] = np.abs(merged_df["lhs"] - merged_df["rhs"])
#merged_df["adaptive_tol"] = 1.0 + 0.01 * merged_df["strike"]
#merged_df["adaptive_tol"] = 5.0 + 0.02 * merged_df["strike"]

#merged_df["parity_clean"] = merged_df["abs_diff"] <= merged_df["adaptive_tol"]

merged_df["relative_error"] = merged_df["abs_diff"] / (merged_df["rhs"].abs() + 1e-6)
merged_df["parity_clean"] = merged_df["relative_error"] < 0.15  # allow up to 15% mismatch


# --- 3. Monotonicity ---
calls_df = calls_df.sort_values("strike").reset_index(drop=True)
puts_df = puts_df.sort_values("strike").reset_index(drop=True)

calls_diff = calls_df["mid"].diff()
puts_diff = puts_df["mid"].diff()

calls_df["monotonic_clean"] = True
puts_df["monotonic_clean"] = True
calls_df.loc[1:, "monotonic_clean"] = calls_diff[1:] <= epsilon
puts_df.loc[1:, "monotonic_clean"] = puts_diff[1:] >= -epsilon

# --- 4. Butterfly Arbitrage ---
def butterfly_violation(df):
    df_sorted = df.sort_values("strike").reset_index(drop=True)
    is_clean = [True] * len(df_sorted)
    for i in range(1, len(df_sorted)-1):
        P1 = df_sorted.loc[i-1, "mid"]
        P2 = df_sorted.loc[i, "mid"]
        P3 = df_sorted.loc[i+1, "mid"]
        if P2 > 0.5 * (P1 + P3):
            is_clean[i] = False
    return is_clean

calls_df["butterfly_clean"] = butterfly_violation(calls_df)
puts_df["butterfly_clean"] = butterfly_violation(puts_df)

# --- 5. Merge all conditions ---
calls_df["is_clean"] = (
    calls_df["monotonic_clean"] &
    calls_df["butterfly_clean"] &
    calls_df["strike"].isin(merged_df[merged_df["parity_clean"]]["strike"])
)

puts_df["is_clean"] = (
    puts_df["monotonic_clean"] &
    puts_df["butterfly_clean"] &
    puts_df["strike"].isin(merged_df[merged_df["parity_clean"]]["strike"])
)

# --- 6. Summary ---
# 1. Parity
clean_parity_strikes = set(merged_df[merged_df["parity_clean"]]["strike"])
n_parity_clean = len(clean_parity_strikes)
n_parity_total = merged_df["strike"].nunique()

# 2. Monotonicity
n_mono_calls = calls_df["monotonic_clean"].sum()
n_mono_puts = puts_df["monotonic_clean"].sum()

# 3. Butterfly
n_butterfly_calls = calls_df["butterfly_clean"].sum()
n_butterfly_puts = puts_df["butterfly_clean"].sum()

# Final clean
n_final_calls = calls_df["is_clean"].sum()
n_final_puts = puts_df["is_clean"].sum()

# --- Print Summary ---
print("=== No-Arbitrage Check Summary ===")
print(f"Put-Call Parity clean: {n_parity_clean} / {n_parity_total}")
print(f"Call monotonicity clean: {n_mono_calls} / {len(calls_df)}")
print(f"Put monotonicity clean: {n_mono_puts} / {len(puts_df)}")
print(f"Call butterfly clean: {n_butterfly_calls} / {len(calls_df)}")
print(f"Put butterfly clean: {n_butterfly_puts} / {len(puts_df)}")
print(f"Calls retained (all clean): {n_final_calls} / {len(calls_df)}")
print(f"Puts retained (all clean): {n_final_puts} / {len(puts_df)}")

=== No-Arbitrage Check Summary ===
Put-Call Parity clean: 50 / 157
Call monotonicity clean: 133 / 180
Put monotonicity clean: 146 / 169
Call butterfly clean: 96 / 180
Put butterfly clean: 95 / 169
Calls retained (all clean): 18 / 180
Puts retained (all clean): 18 / 169


In [9]:
# Define output directory
output_dir = "../data"
os.makedirs(output_dir, exist_ok=True)

# File naming based on ticker
ticker = stock_data.ticker.lower()

# Save files
historical_df.to_csv(f"{output_dir}/historical_data_{ticker}_{ref_date}.csv", index=False)
calls_df.to_csv(f"{output_dir}/calls_data_{ticker}_{ref_date}.csv", index=False)
puts_df.to_csv(f"{output_dir}/puts_data_{ticker}_{ref_date}.csv", index=False)
spot_price_ref_date.to_csv(f"{output_dir}/spot_price_ref_date_{ticker}_{ref_date}.csv", index=False)